In [ ]:
import pandas as pd
import wget
import os
from tqdm import tqdm
from bs4 import BeautifulSoup

In [ ]:
data = pd.read_csv('../dataset/bfn-trending-strip-deduped.tsv', sep='\t')

In [ ]:
# Filter articles with a timestamp_first value after january 1st 2023
data = data[data['timestamp_first'] > '2023-01-01']

# Download all articles
for i in tqdm(range(len(data))):
    if os.path.exists('../dataset/articles/' + data['url'].iloc[i].split('/')[-1]):
        continue
    wget.download(data['url'].iloc[i], out='../dataset/articles/')

In [ ]:
for file in os.listdir('../dataset/articles/')[:100]:
    soup = BeautifulSoup(open(f"../dataset/articles/{file}", "r+"))

    all_text = soup.main.find_all('h1')[0].parent.parent.parent.findChildren(lambda tag: tag.name in ["p", "h1", "h2", "h3", "h4", "h5", "h6"])

    text = ""
    for i in range(len(all_text) - 3):
        if all_text[i].name != 'p' and all_text[i].name != 'span' and len(all_text[i].text.split()) and len(all_text[i].text.split()[0]) > 0:
            text += '#' * int(all_text[i].name[-1]) + ' '
        if len(all_text[i].text.split()) and all_text[i].text.split()[0] != "BuzzFeed":
            text += all_text[i].text + '\n\n'
    
    with open(f"../dataset/processed/{file}.md", "w+") as f:
        f.write(text)
